In [36]:
patch_dict={'op': '<=',
  'opr': [{'state': 'input', 'type': 'int', 'name': 'nick', 'coord': "infer.c:35:27"},
   {'state': 'element',
    'type': 'int',
    'name': '_const',
    'coord': None,
    'const': 5}]}

In [6]:
from instrumentation import *


In [38]:
'''
patch synthesis
insert tmp var heapobject
insert patch (return location)

'''

'\npatch synthesis\ninsert tmp var heapobject\ninsert patch (return location)\n\n'

In [20]:
filelist=read_file("dep__src.c")

In [24]:
patch_cand={'func': {'op': '==',
  'opr': [{'state': 'input',
    'type': 'int',
    'name': 'a',
    'coord': 'dep__src.c:5:14',
    'ret': [{'coord': 'dep__src.c:10:9'},
     {'coord': 'dep__src.c:12:9'},
     {'coord': 'dep__src.c:12:16'}]},
   {'state': 'element',
    'type': 'int',
    'name': '_const',
    'coord': None,
    'const': 5}]}}

In [25]:
def insert_tmp_var(filelist,patch_cand):
    patch={}
    for func,patch_dict in patch_cand.items():
        cur_filelist=filelist.copy()
        retloc=patch_dict["opr"][0]["ret"]
        patch[func]={"patch":None,"ret":retloc}
        if patch_dict["op"]=="1":
            continue
        patchlist=[]
        for opr in patch_dict["opr"]:
            if opr["name"]=="_const":
                patchlist.append(str(opr["const"]))
            else:
                line=int(opr["coord"].split(":")[1])
                state=opr["state"]
                insert_stat=f"int tmp_{opr['name']} = {opr['name']};\n"
                patchlist.append("tmp_"+opr["name"])
                if state=="var":
                    cur_filelist[line-1]=cur_filelist[line-1][:-1]+insert_stat
                elif state=="input":
                    tmp=line-1
                    while len(cur_filelist[tmp])<2 or cur_filelist[tmp][-2]!="{":
                        tmp+=1
                    cur_filelist[tmp]=cur_filelist[tmp][:-1]+insert_stat
                for ret in opr["ret"]:
                    if ret in retloc and line>int(ret["coord"].split(":")[1]):
                        retloc.remove(ret)
        patch[func]["patch"]=patchlist[0]+patch_dict["op"]+patchlist[1]
        patch[func]["filelist"]=cur_filelist
    return patch
            
def insert_heap_object(filelist,o,otype,line):
    insert_stat=f"{otype} tmp_{o} = {o};\n"
    filelist[line-1]=filelist[line-1][:-1]+insert_stat
    return filelist,"tmp_"+o

def cur_patch(patch,o):
    if patch:
        return f"if({patch})free({o});"
    else: 
        return f"free({o});"
    

def insert_patch(filelist,line,patch):
    num=0
    for c in filelist[line-1]:
        if c == " ":
            num+=1
        else:
            break
    filelist[line-1]=filelist[line-1][:num]+patch+filelist[line-1][num:]
    return filelist
    

In [27]:
a="     asdf"
num=0
for c in a:
    if c == " ":
        num+=1
b=a[:num]+"bbb"+a[num:]
b

'     bbbasdf'

In [26]:
insert_tmp_var(filelist,patch_cand)

{'func': {'patch': 'tmp_a==5',
  'ret': [{'coord': 'dep__src.c:10:9'},
   {'coord': 'dep__src.c:12:9'},
   {'coord': 'dep__src.c:12:16'}],
  'filelist': [' \n',
   ' \n',
   ' \n',
   '\n',
   'int func(int a){int tmp_a = a;\n',
   '    int *p=malloc(4);\n',
   '    int *b=&a;\n',
   '    if(a<5){\n',
   '        free(p);\n',
   '        return 0;\n',
   '    }else{int tmp_a = a;\n',
   '        return 1;\n',
   '    }\n',
   '\n',
   '}\n',
   '\n',
   '\n',
   '\n',
   'int LLVMFuzzerTestOneInput(char *data, int size) {\n',
   '    int n=0;\n',
   '    for(int i=0;i<size;i++){\n',
   '        if(! isdigit(*(data+i)))return 0;\n',
   "        n=n*10+ *(data+i)-'0';\n",
   '    }\n',
   '    func(n);\n',
   '    return 0;\n',
   '}\n']}}

In [9]:
filelist

['\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'typedef struct _p11_dict {\n',
 '    struct p11_dictbucket **buckets;\n',
 '}p11_dict;\n',
 '\n',
 'typedef struct p11_dictbucket {\n',
 '    struct p11_dictbucket *next;\n',
 '} dictbucket;\n',
 '\n',
 'p11_dict *\n',
 'p11_dict_new (){\n',
 '    p11_dict *dict;\n',
 '    dict = malloc (sizeof (p11_dict));\n',
 '    if (dict) {\n',
 '        dict->buckets = (dictbucket **)malloc (sizeof (dictbucket *));\n',
 '        if (!dict->buckets) {\n',
 '            free (dict);\n',
 '            return NULL;\n',
 '        }\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 '\n',
 'p11_dict *\n',
 'func(int c){\n',
 '    p11_dict *dict;\n',
 '    dict =p11_dict_new();\n',
 '    if(c==0){\n',
 '        return NULL;\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 'p11_dict *\n',
 'p11_constant_reverse (int nick)\n',
 '{\n',
 '    p11_dict *lookups;\n',
 '    lookups = p11_dict_new ();\n',
 '    if (nick) {\n',
 '        return NULL;\n',
 '    } els

In [59]:
fl,pd=insert_tmp_var(filelist,patch_dict)

In [60]:
insert_heap_object(fl,"lookups"," p11_dict *",38)

(['typedef struct _p11_dict {\n',
  '    struct p11_dictbucket **buckets;\n',
  '}p11_dict;\n',
  '\n',
  'typedef struct p11_dictbucket {\n',
  '    struct p11_dictbucket *next;\n',
  '} dictbucket;\n',
  '\n',
  'p11_dict *\n',
  'p11_dict_new (){\n',
  '    p11_dict *dict;**(*p)->a.b=b+c;\n',
  '    dict = malloc (sizeof (p11_dict));\n',
  '    if (dict) {\n',
  '        dict->buckets = (dictbucket **)malloc (sizeof (dictbucket *));\n',
  '        if (!dict->buckets) {\n',
  '            free (dict);\n',
  '            return NULL;\n',
  '        }\n',
  '    }\n',
  '    return dict;\n',
  '}\n',
  '\n',
  '\n',
  'p11_dict *\n',
  'func(int c){\n',
  '    p11_dict *dict;\n',
  '    dict =p11_dict_new();\n',
  '    if(c==0){\n',
  '        return NULL;\n',
  '    }\n',
  '    return dict;\n',
  '}\n',
  '\n',
  'p11_dict *\n',
  'p11_constant_reverse (int nick)\n',
  '{int tmp_nick = nick;\n',
  '    p11_dict *lookups;\n',
  '    lookups = p11_dict_new (); p11_dict * tmp_lookups = 

In [62]:
patch=cur_patch(pd,"lookups")

In [63]:
insert_patch(filelist,40,patch)

['typedef struct _p11_dict {\n',
 '    struct p11_dictbucket **buckets;\n',
 '}p11_dict;\n',
 '\n',
 'typedef struct p11_dictbucket {\n',
 '    struct p11_dictbucket *next;\n',
 '} dictbucket;\n',
 '\n',
 'p11_dict *\n',
 'p11_dict_new (){\n',
 '    p11_dict *dict;**(*p)->a.b=b+c;\n',
 '    dict = malloc (sizeof (p11_dict));\n',
 '    if (dict) {\n',
 '        dict->buckets = (dictbucket **)malloc (sizeof (dictbucket *));\n',
 '        if (!dict->buckets) {\n',
 '            free (dict);\n',
 '            return NULL;\n',
 '        }\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 '\n',
 'p11_dict *\n',
 'func(int c){\n',
 '    p11_dict *dict;\n',
 '    dict =p11_dict_new();\n',
 '    if(c==0){\n',
 '        return NULL;\n',
 '    }\n',
 '    return dict;\n',
 '}\n',
 '\n',
 'p11_dict *\n',
 'p11_constant_reverse (int nick)\n',
 '{int tmp_nick = nick;\n',
 '    p11_dict *lookups;\n',
 '    lookups = p11_dict_new (); p11_dict * tmp_lookups = lookups;\n',
 '    if (nick) {\n',
 'i